In [1]:
import sys,importlib
sys.path.append('../')
import simulation
from simulation import dropna
from scipy.integrate import odeint
from numpy import *
import plotly.offline as py
import plotly.graph_objs as go
import pandas

In [2]:
delTime = .001
time = 40
Timeline = arange(0.0,time,delTime)
a0,a1 = 1.5,.2
w = 1.3

### duffing master

In [3]:
alpha = 1
delta = .25
gamma = .3
beta = -1
omega = 1
def dufffingOscillator(state,t):
    x,y = state[0],state[1]
    xdot = y
    ydot = -alpha*(x**3) - beta* x - delta*y + gamma*cos(omega*t)
    return asarray((xdot,ydot))

In [4]:
dMaster = odeint(dufffingOscillator, [0,0], Timeline, rtol=1.49012e-6, atol=1.49012e-6)
dMaster = pandas.DataFrame(dMaster,columns=['x','y'])
dMaster['t'] = Timeline

In [5]:
deltaTimeline = dMaster['t'].diff().fillna(delTime).shift(-1)
dMaster['dX'] = dMaster['x'].diff().shift(-1)/deltaTimeline
dMaster['dY'] = dMaster['y'].diff().shift(-1)/deltaTimeline
dMaster = dMaster.fillna(method='ffill')

## synchronisation coupling

In [6]:
a0,a1,w = 3.5,2.5,1*pi
epsilon = 1.5

def zetaInvar(stateA,stateB,t):
    return -asarray([w*a1*cos(w*t)]*len(stateA))
def chi(stateA,stateB,t):
    return stateA-stateB-a0-a1*sin(w*t)
def zetaNull(stateA,stateB,t):
    return asarray([0,0],dtype=float)

### resampling

In [7]:
deltaTime = .001
master = dMaster.iloc[arange(0,len(dMaster),int(deltaTime/delTime))]

In [8]:
timeline = master['t']

## slave evolution

In [9]:
funcX = lambda x,y,t : 0
funcY = lambda x,y,t : -alpha*(x**2)

In [ ]:
master.loc[:,'gX'] = master['dX'] - master.apply(lambda state:funcY(state['x'],state['y'],state['t']),axis=1)
master.loc[:,'gY'] = master['dY'] - master.apply(lambda state:funcY(state['x'],state['y'],state['t']),axis=1)

In [ ]:
def slaveEvolution(master,zeta):
    def slaving(slave,t):
        time_index = int(t/deltaTime)
        stateA = master.iloc[time_index]
        A = stateA[['x','y']].to_numpy()
        gA = stateA[['gX','gY']].to_numpy()
        
        fA = [funcX(A[0],A[1],t),funcY(A[0],A[1],t)]
        dA = gA + array(fA)
        dSlave =  dA + zeta(A,slave,t) + epsilon*chi(A,slave,t)
        return dSlave
    slave0 = zeros((2))
    timeline = master['t'].to_numpy()
    slave = odeint(slaving, slave0, timeline, rtol=1.49012e-6, atol=1.49012e-6)
    slave = pandas.DataFrame(slave,columns=['x','y'])
    return slave

In [ ]:
slave_zeta = slaveEvolution(master,zetaInvar)
slave_null = slaveEvolution(master,zetaNull)
D = a0+a1*sin(w*master['t'])
slave_truth = master[['x','y']] - expand_dims(D,axis=1)

In [ ]:
master_trace = go.Scatter(name='master',x=master['x'],y=master['y'],mode='lines')
slave_zeta_trace = go.Scatter(name='slave_zeta',x=slave_zeta['x'],y=slave_zeta['y'],mode='lines')
slave_null_trace = go.Scatter(name='slave_null',x=slave_null['x'],y=slave_null['y'],mode='lines')
slave_truth_trace = go.Scatter(name='true',x=slave_truth['x'],y=slave_truth['y'],mode='lines')
py.iplot([master_trace,slave_zeta_trace,slave_null_trace,slave_truth_trace])

In [ ]:
error_zeta = simulation.syncError(slave_truth[['x','y']].to_numpy(),slave_zeta[['x','y']].shift(0).to_numpy())
error_null = simulation.syncError(slave_truth[['x','y']].to_numpy(),slave_null[['x','y']].shift(0).to_numpy())

In [ ]:
print('error zeta:',dropna(error_zeta.sum(axis=1)).mean())
print('error null:',dropna(error_null.sum(axis=1)).mean())

-------------------